In [29]:
import numpy as np
from pysrc.optimize import make_constants

@make_constants({'sqrt': np.sqrt})
def qf(a,b,c):
    return (-b+sqrt(b**2-4*a*c))/(2*a)

class AcidEquilibrium():
    def __init__(self, ka, c2, c3):
        self.ka = ka
        self.c2 = c2
        self.c3 = c3
        
    def pH(self, c1):
        a = 1
        b = self.c2 + self.c3 + self.ka
        c = self.c2 * self.c3 - c1 * self.ka
        x = qf(a,b,c)
        self.c2 += x
        self.c3 += x
        
        if self.c2 < 0:
            self.c2 = 0
        if self.c3 < 0: 
            self.c3 = 0
        
        return -np.log10(self.c2)
        

        

In [30]:
a = AcidEquilibrium(4.45e-7, 1e-7, 2.02/84)

In [31]:
hcp = 3.4e-2
a.pH(0*hcp)

11.732718352753357

In [33]:
from matplotlib import pyplot as plt
%matplotlib

Using matplotlib backend: TkAgg


In [34]:
z = np.linspace(0, 1) * 3.4e-2

def equil(c1, c2, c3, ka):
    a = 1
    b = c2+c3+ka
    c = c2*c3 - ka*c1
    x = qf(a,b,c)
    return c1-x, c2+x, c3+x

c1 = z
c2 = 1e-7
c3 = 2.02/84
ka = 4.45e-7

z, y, x = equil(c1, c3, c3, ka)

In [39]:
plt.plot(z,y, label="H+ v CO2")
plt.plot(x*100,y, label="HCO3 v CO2")
#plt.gca().set_xscale('log')
plt.legend()

In [42]:
dxdz = np.gradient(ka/y)

plt.plot(z, dxdz)


In [72]:
c1 = .4 *3.4e-2
c2 = 1e-7
c3 = 2.02/84
e1, e2, e3 = equil(c1, c2, c3, ka)
e1-c1, e2 - c2, e3 - c3

(-1.5166293303517375e-07, 1.5166293303517375e-07, 1.5166293303517375e-07)

In [73]:
x1, x2, x3 = equil(c1, c2, c3, ka)
e1, e2, e3 = equil(1*3.4e-2, x2, x3, ka)
e1-x1, e2 - x2, e3 - x3

(0.020399774178378429, 3.7748455460583086e-07, 3.7748455460409613e-07)

In [54]:
c1 = .4 *3.4e-2
c2 = 1e-7
c3 = 2.02/84
x1, x2, x3 = equil(c1, c2, c3, ka)
dc1 = -.2 * 3.4e-2
e1, e2, e3 = equil(x1+dc1, x2, x3, ka)
x2 - e2

1.2582988288302455e-07

In [84]:
from matplotlib.ticker import FormatStrFormatter
co2 = np.linspace(0, .99, 100)
dco2 = 0.2
hco3 = 2.02/84
h = 1e-7
data = [equil(co2 *3.4e-2, 1e-7, hco3, ka) for co2 in np.linspace(0, 1-dco2, 100)]
data2 = []
for c, h, b in data:
    c2 = c + dco2 * 3.4e-2
    e1, e2, e3 = equil(c2, h, b, ka)
    dh = e2-h
    db = e3-b
    data2.append((c, dh, db))
    
x, dh, db = list(zip(*data2))
plt.plot(x, dh)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter("%s"))

In [82]:
dh = np.array(dh)
db = np.array(db)
dh == db
dh

array([  1.25831200e-07,   1.25831146e-07,   1.25831093e-07,
         1.25831040e-07,   1.25830987e-07,   1.25830934e-07,
         1.25830880e-07,   1.25830827e-07,   1.25830774e-07,
         1.25830721e-07,   1.25830668e-07,   1.25830614e-07,
         1.25830561e-07,   1.25830508e-07,   1.25830455e-07,
         1.25830402e-07,   1.25830348e-07,   1.25830295e-07,
         1.25830242e-07,   1.25830189e-07,   1.25830136e-07,
         1.25830082e-07,   1.25830029e-07,   1.25829976e-07,
         1.25829923e-07,   1.25829870e-07,   1.25829816e-07,
         1.25829763e-07,   1.25829710e-07,   1.25829657e-07,
         1.25829604e-07,   1.25829550e-07,   1.25829497e-07,
         1.25829444e-07,   1.25829391e-07,   1.25829338e-07,
         1.25829284e-07,   1.25829231e-07,   1.25829178e-07,
         1.25829125e-07,   1.25829072e-07,   1.25829018e-07,
         1.25828965e-07,   1.25828912e-07,   1.25828859e-07,
         1.25828806e-07,   1.25828752e-07,   1.25828699e-07,
         1.25828646e-07,

In [86]:
def ph(c1, c2, c3, ka=4.45e-7):
    e1, e2, e3 = equil(c1*3.4e-2, c2, c3, ka)
    return - np.log10(e2)

In [88]:
f=1
ph(0, 1e-7, 2.02/84/f)

11.732718352753357

In [89]:
for f in range(10):
    f = 10**-f
    bc = 2.02 * f
    p = ph(0, 1e-7, bc/84)
    print(bc, p)

2.02 11.7327183528
0.202 10.7327743756
0.0202 9.73333502121
0.00202 8.73893089946
0.000202 7.79346876715
2.0200000000000003e-05 7.16692834949
2.0199999999999997e-06 7.01889659585
2.0199999999999998e-07 7.0019136091
2.0200000000000002e-08 7.00019160165
2.02e-09 7.00001916257


In [90]:
ph(0, 1e-7, 0)

7.0

In [ ]:
c = []
co2 = 0
for co2 in range(100):
    co2 /= 100
    co2 *= 3.4e-2
    p = ph(co2, )